In [0]:
from psaw import PushshiftAPI
import datetime
import pandas as pd

api = PushshiftAPI()

today = datetime.datetime.now()
d = datetime.timedelta(days = 1)
yesterday = today - d
yesterday = yesterday.timestamp()
start_epoch = int(yesterday)


hot_subreddit= api.search_submissions(after=start_epoch,
                            subreddit='Bitcoincash',
                            sort_type='score', sort='desc',
                            filter=['title','score','selftext'],
                            limit=1000)

topics_dict = {"title":[],\
               "score":[],\
               "date":[],\
               "body":[]}

    
for submission in hot_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["date"].append(submission.created_utc)
    topics_dict["body"].append(submission.selftext)   


topics_data = pd.DataFrame(topics_dict)

df = topics_data

2021-02-25 00:07:37.004542
 title ... body
0 Cathie Wood Ark CEO: Bitcoin is the "Best Hedg... ... 
1 "When we are all paid in bitcoin, no one can t... ... Russel Okung is an NFL player who signed a con...
2 Jim Cramer Tells GameStop to Buy Bitcoin Amid ... ... 
3 Coinbase publicly reveals it has held Bitcoin ... ... 
4 SAYLOR will keep buying more Bitcoin... MicroS... ... 
5 Today is my birthday and this is my family's g... ... 
6 Sharing some Bitcoin related art I made. 😊 ... 
7 Yellen is positive news for Bitcoin ... Yellen continues to warn the public and inves...
8 Daily Discussion, February 25, 2021 ... Please utilize this sticky thread for all gene...
9 Saylor Advice to No-Coiners ... 
10 Keeps me safe during corrections ... 
11 I made a thing! A website that tells you appro... ... Hey everyone,\n\nI made a quick little website...
12 Every day someone here asks how to buy bitcoin... ... There's nothing in KYC law requiring you to be...
13 Gleec Announces New Free Top-Up Bitcoin Visa C... ... [deleted]
14 I run a Insulation business. I mostly do blown... ... Thank you!
15 Believe It or Not: Bitcoin has become a Good L... ... 
16 Good article: Janet Yellen Sounds Like She’s S... ... 
17 Bitcoin purchase BLOCKED by a bank! ... Hey everyone, \n\nJust thought I’d share this ...
18 Discussions - 124 BTC stolen from users of Bit... ... 
19 Silent Generation worse than Boomers when it c... ... Steve Hanke, a 78 year old professor at Johns ...
20 Excited to announce that we are raising $1b to... ... [deleted]
21 With Charlie Munger, Bill Gates, and Janet Yel... ... 
22 Small business making move into BTC ... I own a small business and recently moved most...
23 All In ... Gas prices - increasing\n\nUSD - decreasing\n\...
24 Be Your Own Bank Art Print - "It’s not your mo... ... 
25 BTC Scams. Beware. Don’t invest. ... SCAM INVESTMENT ❌\n\nLIST OF SCAM SITES\n\n-TR...
26 When Bitcoin Loses 20% in a Few Hours, Underst... ... 
27 Proof of Work is Efficient — Dan Held ... 
28 PayPal Europe still hasn't entered the Bitcoin... ... *This is a reminder that along with the huge i...
29 Bitcoin Private Key - Flipping a Coin 256 Times ... 
.. ... ... ...
276 Join the waitlist for the BlockFi credit card ... ... 
277 I have been TRYING for a WEEK to buy btc on cr... ... Hey! So for the past 5 days I have been trying...
278 Buy Bitcoin (BTC) with card instantly. No ID. ... [removed]
279 Anyone else feel like most of the politicians ... ... 
280 Buy Bitcoin (BTC) with card instantly. No ID. ... [removed]
281 Join the waitlist for the BlockFi credit card ... ... 
282 Unesco e Chainlink ... [removed]
283 Bitcoin vs Nano. Best arguments against Nano c... ... [removed]
284 I see a 3-bedroom house in the burbs in the la... ... [deleted]
285 Janet Yellen Sounds Like She’s Scared Of Bitcoin ... 
286 Every time ... [deleted]
287 Just jumped in!! ... 
288 Bitcoin Has No Intrinsic Value, Asset Is Too V... ... 
289 We've all experienced the crypto sell-off earl... ... 
290 Come What may ... [removed]
291 Every time ... [deleted]
292 Upside of the dip ... [removed]
293 Largest BTC purchase in history just announced ... 
294 Facts ... 
295 $BITCOIN ... [removed]
296 Join the waitlist for the BlockFi credit card ... ... 
297 Talk/Hold/Buy/and kick bitcoin to the moon ... [removed]
298 COMP has true potential because of what they d... ... 
299 Bitcoin fees ridiculously high? ... So I can’t withdraw my bitcoin to Binance to s...
300 Bitcoin price crashed! 🚨🚨Here is what comes ne... ... [deleted]
301 Bitcoin What is next? ... We've all experienced the crypto sell-off earl...
302 Doubling coin ... [deleted]
303 Is Bitcoin revolutionary? Does nothing to chan... ... Some of the excitement about Bitcoin is that i...
304 New to bitcoin quick question ... [removed]
305 Janet Yellen Sounds Like She’s Scared Of Bitcoin ... 

[306 rows x 4 columns]

In [0]:

sent_score_list = []

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
nltk.download('punkt')

analyzer = SentimentIntensityAnalyzer()
for paragraph in df["body"]:
      
    
    sentence_list = tokenize.sent_tokenize(paragraph)
    paragraphSentiments = 0.0
    for sentence in sentence_list:
        vs = analyzer.polarity_scores(sentence)
        paragraphSentiments += vs["compound"]

    try:
        sent_score = round(paragraphSentiments / len(sentence_list), 3)
        sent_score_list.append(sent_score)
    except:
        sent_score = 0
        sent_score_list.append(sent_score)


title_score_list = []

for title in df["title"]:
    vs = analyzer.polarity_scores(title)
    title_score = vs["compound"]
    title_score_list.append(title_score)
    



df['title sentiment score'] = title_score_list
df['body sentiment score'] = sent_score_list

df['Date'] = pd.to_datetime(df['date'],unit='s')

df['Date'] = df['Date'].dt.strftime('%Y/%m/%d')
df['Date'] = pd.to_datetime(df.Date)


data_load = df[['Date','title sentiment score','body sentiment score','title','body','score']]


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
Empty DataFrame
Columns: [Date, title sentiment score, body sentiment score, title, body, score]
Index: []

In [0]:
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import json


#setup connection (information like this should be stored in secure place like KeyVault)
client = cosmos_client.CosmosClient("https://trade-parser.documents.azure.com:443/","{API key}")
db = client.create_database_if_not_exists(id= "BCHSentimentDatabase")

container = db.create_container_if_not_exists(id="APIparsedBCHreddit",partition_key=PartitionKey(path ='/date',kind ='Hash'))


# load data to nosql

for i in range(0,data_load.shape[0]):
    
    # create a dictionary for the selected row
    data_dict = dict(data_load.iloc[i,:])
    # convert the dictionary to a json object.
    data_dict = json.dumps(data_dict, default =str)
    updated_item = container.upsert_item(json.loads(data_dict))
    updated_item